# Parallelization Workflow

In [1]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("dev.langchain4j:langchain4j:1.0.0-beta2")
@file:DependsOn("dev.langchain4j:langchain4j-anthropic:1.0.0-beta2")

In [2]:
val apiKey = System.getenv("ANTHROPIC_API_KEY")

In [3]:
import dev.langchain4j.data.message.SystemMessage.systemMessage
import dev.langchain4j.data.message.UserMessage.userMessage
import dev.langchain4j.model.anthropic.AnthropicChatModel
import dev.langchain4j.model.anthropic.AnthropicChatModelName
import dev.langchain4j.model.chat.chat

suspend fun llmCall(
    prompt: String,
    systemPrompt: String? = null,
    model: AnthropicChatModelName = AnthropicChatModelName.CLAUDE_3_7_SONNET_20250219
): String {
    val client = AnthropicChatModel.builder()
        .apiKey(apiKey)
        .modelName(model)
        .maxTokens(4096)
        .temperature(0.1)
        .build()

    return withContext(Dispatchers.IO) {
        val response = client.chat {
            systemPrompt?.let { messages += systemMessage(it) }
            messages += userMessage(prompt)
        }
        response.aiMessage().text()
    }
}

In [4]:
/**
 * Process multiple inputs concurrently with the same prompt.
 */
suspend fun parallel(prompt: String, inputs: List<String>, nWorkers: Int = 3): List<String> = coroutineScope {
    // Create a dispatcher with a fixed thread pool
    val dispatcher = Dispatchers.IO.limitedParallelism(nWorkers)

    // Launch a coroutine for each input and collect the results
    inputs.map { input: String ->
        async(dispatcher) { llmCall("$prompt\nInput: $input") }
    }.awaitAll()
}

In [5]:
val stakeholders = listOf(
    """
    Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns
    """,

    """
    Employees:
    - Job security worries
    - Need new skills
    - Want clear direction
    """,

    """
    Investors:
    - Expect growth
    - Want cost control
    - Risk concerns
    """,

    """
    Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions
    """
)

In [7]:
runBlocking {
    val impactResults = parallel(
        """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
        stakeholders
    )


    impactResults.forEach {
        print(it)
    }
}

# Market Impact Analysis for Price-Sensitive, Tech-Focused, Environmentally Conscious Customers

## Key Market Changes & Impacts

### HIGH PRIORITY: Price Sensitivity Challenges
* **Rising inflation** will amplify price sensitivity, potentially reducing purchasing power
* **Competitive pricing landscape** may create expectations for discounts or promotions
* **Economic uncertainty** could lead to delayed purchasing decisions

### HIGH PRIORITY: Technology Expectations
* **Rapid tech innovation cycles** creating higher baseline expectations
* **Increased availability** of advanced features at lower price points from competitors
* **Digital integration** becoming standard rather than premium feature

### MEDIUM PRIORITY: Environmental Considerations
* **Growing regulatory pressures** around sustainability and carbon footprint
* **Increased consumer awareness** of environmental impacts in purchasing decisions
* **Social pressure** to align with environmentally responsible brands

## Recom